In [1]:
#imports
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
from torch import nn
import pickle
from torch.utils.tensorboard import SummaryWriter
import os
import shutil
import numpy as np
#project specific imports
from modules.model import FOMOnet
from modules.transcripts import Transcripts
from modules.utils import *

2023-04-24 14:37:24.170287: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 14:37:24.193252: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 14:37:24.554101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#create runs directory for curves visualization with tensorboard
#shutil.rmtree(f'runs/', ignore_errors=True)
#os.mkdir('runs')
writer = SummaryWriter()
test_writer = SummaryWriter()

In [3]:
#load train & test data
X_train, y_train = pickle.load(open('data/train_split0.pkl', 'rb'))
X_test, y_test = pickle.load(open('data/test_split0.pkl', 'rb'))

In [4]:
#pre-processing data for pytorch DataLoader
train_set = Transcripts(X_train, y_train)
test_set = Transcripts(X_test, y_test)

In [5]:
#hyperparameters
batch_size = 8
epochs = 30
lr = 0.0005

In [6]:
#create DataLoader object for train & test data
train_loader = DataLoader(train_set, batch_size=batch_size, collate_fn=pack_seqs, shuffle=True, num_workers=24)
test_loader = DataLoader(test_set, batch_size=batch_size, collate_fn=pack_seqs, shuffle=True, num_workers=24)

In [7]:
#instantiate model, optimizer and loss function
fomonet = FOMOnet(num_channels=4).cuda()
optimizer = optim.Adam(fomonet.parameters(), lr)
loss_function = nn.BCELoss(reduction='none').cuda()

In [8]:
#train model
best_model = 1.0
for epoch in range(epochs):
    fomonet.train()
    for batch in train_loader:
        X = batch[0].view(len(batch[0]),1,-1).cuda()
        y = batch[1].view(len(batch[1]),-1).cuda()
        X_one_hot = batch[2].view(len(batch[0]),4,-1).cuda()
        outputs = fomonet(X_one_hot).view(len(batch[0]),-1)
        fomonet.zero_grad()
        loss = get_loss(outputs, X, y, loss_function)
        loss.backward()
        optimizer.step()
        loss = loss.cpu().detach().numpy()
        writer.add_scalar("Loss/train", loss, epoch)
    fomonet.eval()
    test_losses = []
    for batch in test_loader:
        X = batch[0].view(len(batch[0]),1,-1).cuda()
        y = batch[1].view(len(batch[1]),-1).cuda()
        X_one_hot = batch[2].view(len(batch[0]),4,-1).cuda()
        outputs = fomonet(X_one_hot).view(len(batch[0]),-1)
        test_loss = get_loss(outputs, X, y, loss_function)
        test_loss = test_loss.cpu().detach().numpy()
        test_losses.append(test_loss)
        test_writer.add_scalar("Loss/test", test_loss, epoch)
    print(f'{epoch}_{np.mean(test_losses)}')
    if np.mean(test_losses) < best_model:
        best_model = np.mean(test_losses)
        torch.save(fomonet.state_dict(), f'trained_models#fomonet.pt')

0_0.3498176336288452
1_0.2326512485742569
2_0.10621431469917297
3_0.08549068868160248
4_0.07314513623714447
5_0.06378301978111267
6_0.06564688682556152
7_0.061877064406871796
8_0.05840793624520302
9_0.056987836956977844
10_0.0544811449944973
11_0.05996118485927582
12_0.0569026917219162


KeyboardInterrupt: 

In [ ]:
writer.flush()
test_writer.flush()
writer.close()
test_writer.close()